## COVID-19

The **coronavirus (COVID-19)** pandemic is the greatest global humanitarian challenge the world has faced since World War II. The pandemic virus has spread widely, and the number of cases is rising daily. The government is working to slow down its spread.

'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`

   ![Coronavirus particle Image](https://media.istockphoto.com/vectors/corona-virus-covid19-banner-illustration-protective-medical-mask-and-vector-id1224508827?k=6&m=1224508827&s=170667a&w=0&h=fytt2oRlwlpfjuo_n3Nq9HE5xvgYyWIoRBAetJCIjac=)

####Importing the libraries

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime

raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])

In [4]:
df.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': None, 'Count...","{'x': 67.70995300000004, 'y': 33.93911000000003}"
1,"{'OBJECTID': 2, 'Province_State': None, 'Count...","{'x': 20.168300000000045, 'y': 41.15330000000006}"
2,"{'OBJECTID': 3, 'Province_State': None, 'Count...","{'x': 1.6596000000000686, 'y': 28.033900000000..."
3,"{'OBJECTID': 4, 'Province_State': None, 'Count...","{'x': 1.5218000000000416, 'y': 42.50630000000007}"
4,"{'OBJECTID': 5, 'Province_State': None, 'Count...","{'x': 17.87390000000005, 'y': -11.202699999999..."


In [5]:
df["attributes"][0]

{'Active': 3614,
 'Admin2': None,
 'Combined_Key': 'Afghanistan',
 'Confirmed': 58542,
 'Country_Region': 'Afghanistan',
 'Deaths': 2565,
 'FIPS': None,
 'Last_Update': 1619245241000,
 'Lat': 33.93911,
 'Long_': 67.709953,
 'OBJECTID': 1,
 'Province_State': None,
 'Recovered': 52363}

####Cleaning Data

In [6]:
data_list = df["attributes"].tolist()
df_final = pd.DataFrame(data_list)
df_final.set_index("OBJECTID")
df_final = df_final[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Active", "Last_Update"]]

In [7]:
df_final.head()

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,Last_Update
0,Afghanistan,None,33.93911,67.709953,58542,2565,52363,3614,1.619245e+12
1,Albania,None,41.15330,20.168300,130270,2367,104278,23625,1.619245e+12
2,Algeria,None,28.03390,1.659600,120562,3190,84038,33334,1.619245e+12
3,Andorra,None,42.50630,1.521800,13007,123,12423,461,1.619245e+12
4,Angola,None,-11.20270,17.873900,25279,574,23089,1616,1.619245e+12


####To Convert Time 

In [8]:
def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)

df_final = df_final.dropna(subset=["Last_Update"])
df_final["Province_State"].fillna(value="", inplace=True)

df_final["Last_Update"]= df_final["Last_Update"]/1000
df_final["Last_Update"] = df_final["Last_Update"].apply(convertTime)

df_final.head()

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,Last_Update
0,Afghanistan,,33.93911,67.709953,58542,2565,52363,3614,2021-04-24 06:20:41
1,Albania,,41.15330,20.168300,130270,2367,104278,23625,2021-04-24 06:20:41
2,Algeria,,28.03390,1.659600,120562,3190,84038,33334,2021-04-24 06:20:41
3,Andorra,,42.50630,1.521800,13007,123,12423,461,2021-04-24 06:20:41
4,Angola,,-11.20270,17.873900,25279,574,23089,1616,2021-04-24 06:20:41


####Aggregating data

In [9]:
df_total = df_final.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum",
        "Active" : "sum"
    }
)

df_total = df_total.sort_values("Confirmed", ascending=False)

df_total.head()

,Country_Region,Confirmed,Deaths,Recovered,Active
178,US,31992070,571199,0,0
79,India,16610481,189544,13867997,2552940
23,Brazil,14237078,386416,12586272,1264390
62,France,5502014,102655,336234,5063125
142,Russia,4691290,105718,4321477,264095


In [10]:
total_confirmed = df_final["Confirmed"].sum()
total_recovered = df_final["Recovered"].sum()
total_deaths = df_final["Deaths"].sum()
total_active = df_final["Active"].sum()

In [11]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

df_top10 = df_total.nlargest(10, "Active")
top10_countries_4 = df_top10["Country_Region"].tolist()
top10_active= df_top10["Active"].tolist()

In [12]:
figa = make_subplots(
    rows = 1, cols = 4,

    specs=[
            [{"type": "indicator", "colspan":1}, {"type": "indicator", "colspan":1}, {"type": "indicator", "colspan":1}, {"type": "indicator", "colspan":1}],
          ]
)

figa.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=1
)

figa.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=2
)

figa.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths, 
        title="Death Cases",
    ),
    row=1, col=3
)

figa.add_trace(
    go.Indicator(
        mode="number",
        value=total_active, 
        title="Active Cases",
    ),
    row=1, col=4
)
figa.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Status (Last Updated: " + str(df_final["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h"
)
figa.write_html('2.html', auto_open=True)

In [13]:
figa.show()

####Bar Graph

In [14]:
figb = make_subplots(
    rows = 4, cols = 3,

    specs=[
            [ {"type": "bar", "colspan":3}, None, None],
            [ {"type": "bar", "colspan":3}, None, None],
            [ {"type": "bar", "colspan":3}, None, None],
            [ {"type": "bar", "colspan":3}, None, None],
          ]
)

figb.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=1, col=1
)

figb.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=2, col=1
)

figb.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=3, col=1
)
figb.add_trace(
    go.Bar(
        x=top10_countries_4,
        y=top10_active, 
        name= "Active Cases",
        marker=dict(color="orangered"), 
        showlegend=True),
    row=4, col=1
)

figb.update_layout(
    template="plotly_dark",
    title = "Bar Plot ",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.425, y=1.05),

)

figb.write_html('3.html', auto_open=True)


In [15]:
figb.show()

In [16]:
figc = make_subplots(
    rows = 4, cols = 3,

    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None ],
            [    None, None, None  ],
            [    None, None, None  ],
            [    None, None, None  ],
          ]
)


message = df_final["Country_Region"] + " " + df_final["Province_State"] + "<br>"
message += "Confirmed: " + df_final["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_final["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_final["Recovered"].astype(str) + "<br>"
message += "Active: " + df_final["Active"].astype(str) + "<br>"
message += "Last updated: " + df_final["Last_Update"].astype(str)
df_final["text"] = message

figc.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = df_final["Long_"],
        lat = df_final["Lat"],
        hovertext = df_final["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = df_final['Confirmed'],
            cmax = df_final['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = 0.07
        )

    ),
    
    row=1, col=1
)

figc.update_layout(
    template="plotly_dark",
    title = "Scatter Geo ",

    geo = dict(
            projection_type="natural earth",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lakecolor="LightBlue"
    ),

    annotations=[
        dict(
            text="Source: https://bit.ly/3aEzxjK",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.55,
            y=-0.1)
    ]
)

figc.write_html('4.html', auto_open=True)

In [17]:
figc.show()

In [18]:
!pip install jupyter-dash

     |████████████████████████████████| 81kB 8.8MB/s 
     |████████████████████████████████| 1.0MB 40.0MB/s 
     |████████████████████████████████| 3.5MB 41.7MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 1.8MB 49.2MB/s 
     |████████████████████████████████| 358kB 47.3MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85834 sha256=f5dda51766f7d4d67ab2966b3bfe6e4011bc3b89bd41d4ac70ff214b24dca415
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014875 sha256=bbe001262d648948530379ae301174b05f0db808a56134a8f8014f541f3776dd
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540993 sha256=2b7fd77678eb3a3bf0c16

In [19]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

In [21]:
import plotly.graph_objects as go # or plotly.express as px
import base64
fig1 = go.Figure(figa)   # or any Plotly Express function e.g. px.bar(...)
fig2 = go.Figure(figb)
fig3 = go.Figure(figc)

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Covid 19 Dashboard  "),
    html.H2("Covid 19 "),
    html.P("The **coronavirus (COVID-19)** pandemic is the greatest global humanitarian challenge the world has faced since World War II. The pandemic virus has spread widely, and the number of cases is rising daily. The government is working to slow down its spread. 'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named SARS-CoV-2 by the World Health Organization (WHO). 3The disease it causes has been named “coronavirus disease 2019” (or “COVID-19”).`"),
    dcc.Graph(figure=fig1),
    dcc.Graph(figure=fig2),
    dcc.Graph(figure=fig3),
])

In [22]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>